# BasicRobotInteraction

## Interaction with the Virtual Robot
The virtual robot is described in 
<a href="http://htmlpreview.github.com/?https://github.com/anatali/iss2020Lab/blob/master/it.unibo.robot2020/userDocs/LabVirtualRobot.html" target="web">
LabVirtualRobot</a>

In [ ]:
##############################################################
# clientToWenv.py
# sendToVirtualRobot : sends a command in output
# read               : acquires data from input
##############################################################
import socket
import time
import json

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
virtualRobotPort = 8999
sep = ';'   #required by the Soffritti virtual robot 'language'

goForwardMsg  = '{"type": "moveForward",   "arg": -1  }'
goBackwardMsg = '{"type": "moveBackward",  "arg": -1  }'
turnLeftMsg   = '{"type": "turnLeft",      "arg": 400 }'
turnRightMsg  = '{"type": "turnRight",     "arg": 400 }'
haltMsg       = '{"type": "alarm",         "arg": 0   }'

def connect(port) :
    server_address = ('localhost', port)
    sock.connect(server_address)    
    print("CONNECTED WITH the virtual robot" , server_address)

def sendToVirtualRobot( message ) :
    print("forward ", message)
    msgJson = json.loads( message )
    msg     = sep+ json.dumps(msgJson)+ sep 
    byt=msg.encode()    #required in Python3
    sock.send( byt )

def work() :
    sendToVirtualRobot( goForwardMsg ) 
    time.sleep(1)
    sendToVirtualRobot( haltMsg ) 

def read() :
    BUFFER_SIZE = 1024
    data = sock.recv(BUFFER_SIZE)
    print( "received data:", data )

def terminate() :
    sock.close()
    print("BYE")

###########################################    
connect(virtualRobotPort)
work()
read()
terminate()

## Interaction with the basicrobot qactor

In [2]:
##############################################################
# sendDispatch       : sends a command in output
# read               : acquires data from input
##############################################################
import socket
import time

basicRobotPort = 8020
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

goForwardMsg  = "msg(cmd,dispatch,python,basicrobot,cmd(w),1)"
goBackwardMsg = "msg(cmd,dispatch,python,basicrobot,cmd(s),1)" 
turnLeftMsg   = "msg(cmd,dispatch,python,basicrobot,cmd(a),1)"  
turnRightMsg  = "msg(cmd,dispatch,python,basicrobot,cmd(d),1)"  
haltMsg       = "msg(cmd,dispatch,python,basicrobot,cmd(h),1)"

def connect(port) :
    server_address = ('localhost', port)
    sock.connect(server_address)    
    print("CONNECTED WITH basicrobot" , server_address)

def sendDispatch( message ) :
    print("forward ", message)
    msg = message + "\n"
    byt=msg.encode()    #required in Python3
    sock.send(byt)

def work() :
    sendDispatch( goForwardMsg ) 
    time.sleep(3)
    sendDispatch( haltMsg )
    time.sleep(1)

def read() :
    BUFFER_SIZE = 1024
    data = sock.recv(BUFFER_SIZE)
    print( "received data:", data )

def terminate() :
    #sock.close()    #qak infrastr receives a msg null
    print("BYE")

###########################################    
connect(basicRobotPort)
work()
terminate()    

CONNECTED WITH basicrobot ('localhost', 8020)
forward  msg(cmd,dispatch,python,basicrobot,cmd(w),1)
forward  msg(cmd,dispatch,python,basicrobot,cmd(h),1)
BYE


## Interaction with the robotmind qactor

In [ ]:
import socket
import time

basicRobotPort = 8020
mindRobotPort  = 8035

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

def connect(port) :
    server_address = ('localhost', port)
    sock.connect(server_address)    
    print("CONNECTED WITH basicrobot" , server_address)

def forward( message ) :
    print("forward ", message)
    msg = message + "\n"
    byt=msg.encode()    #required in Python3
    sock.send(byt)

def work() :
    forward( "msg(cmd,dispatch,python,basicrobot,cmd(w),1)" ) 
    time.sleep(1)
    forward("msg(cmd,dispatch,python,basicrobot,cmd(h),1)")

def read() :
    BUFFER_SIZE = 1024
    data = sock.recv(BUFFER_SIZE)
    print( "received data:", data )

def oneStep() :
    forward( "msg(onestep,dispatch,python,onestepahead,onestep(1000),1)" ) 

def terminate() :
    sock.close()
    print("BYE")

###########################################    
connect(mindRobotPort)
#work()
oneStep()
terminate()

In [ ]:
import time
import paho.mqtt.client as paho

brokerAddr="localhost"

def on_message(client, userdata, message) :   #define callback
    evMsg = str( message.payload.decode("utf-8")  )
    print("evMsg=", evMsg  )
    
client= paho.Client("receiver")      
client.on_message=on_message            # Bind function to callback

client.connect(brokerAddr)              #connect
print("connected to broker ", brokerAddr)
print("subscribing to unibo/qak/events")
client.subscribe("unibo/qak/events")      #subscribe
    
print("collecting values; please wait ..." )
client.loop_start()             #start loop to process received messages
time.sleep(30)
client.disconnect()             #disconnect
print("bye")
client.loop_stop()              #stop loop        
    